In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import base64
from io import BytesIO

# =============================================================
#   CONFIGURACIÓN INICIAL DE LA APP
# =============================================================
st.set_page_config(page_title="Altas y Cambios de Beneficiarios", layout="wide")

# Logo
st.markdown(
    """
    <div style="background: linear-gradient(90deg, #0666a3, #27c3a8); padding: 20px; border-radius: 10px; text-align: center;">
        <img src="data:image/png;base64,{}" width="200">
        <h1 style="color:white; margin-top:10px;">Altas y Cambios de Beneficiarios</h1>
        <h3 style="color:white;">Somos tu aliado en protección</h3>
    </div>
    """.format(base64.b64encode(open("/mnt/data/f2f98b15-0813-4e3a-a6d8-6f26349204b4.png", "rb").read()).decode()),
    unsafe_allow_html=True
)

st.write("## Subir archivos para procesar")

archivo_semana = st.file_uploader("📌 Archivo de la semana (ALTAS)", type=["xlsx"])
archivo_bitacora = st.file_uploader("📘 Archivo histórico (bitácora)", type=["xlsx"])

# =============================================================
# Funciones auxiliares (copiadas de tu código original)
# =============================================================
import unicodedata
import difflib
from datetime import datetime
from dateutil.relativedelta import relativedelta

def limpiar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = str(texto).strip()
    texto = ''.join(c for c in unicodedata.normalize('NFKD', texto) if not unicodedata.combining(c))
    texto = texto.replace("Ñ", "¥").replace("ñ", "¥")
    return texto.upper()

def limpiar_certificado(valor):
    if pd.isna(valor):
        return ""
    try:
        return str(int(float(valor)))
    except:
        return str(valor).strip()

def limpiar_fecha_str(valor):
    if pd.isna(valor) or str(valor).strip() == "":
        return ""
    try:
        fecha = pd.to_datetime(str(valor).strip(), errors="coerce")
        if pd.isna(fecha):
            return ""
        return fecha.strftime("%Y-%m-%d")
    except:
        return ""

def calcular_edad(fecha):
    if pd.isna(fecha) or fecha == "":
        return None
    try:
        fecha = pd.to_datetime(fecha)
        hoy = datetime.today()
        return hoy.year - fecha.year - ((hoy.month, hoy.day) < (fecha.month, fecha.day))
    except:
        return None

def nombre_completo(df):
    return (
        df["NOMBRE"].astype(str).str.strip() + " " +
        df["APELLIDO_PATERNO"].astype(str).str.strip() + " " +
        df["APELLIDO_MATERNO"].astype(str).str.strip()
    ).str.replace(r"\s+", " ", regex=True)

def crear_ids(df):
    df["CERTIFICADO_NOMBRE"] = df["CERTIFICADO"].astype(str) + "_" + df["NOMBRE"].astype(str)
    df["CERTIFICADO_NOMBRE_COMPLETO"] = df["CERTIFICADO"].astype(str) + "_" + df["NOMBRE_COMPLETO"].astype(str)
    df["CERTIFICADO_FECHA"] = df["CERTIFICADO"].astype(str) + "_" + df["FECHA_DE_NACIMIENTO_BENEFICIARIO"].astype(str)
    df["CERTIFICADO_APELLIDO_PATERNO"] = df["CERTIFICADO"].astype(str) + "_" + df["APELLIDO_PATERNO"].astype(str)
    df["CERTIFICADO_APELLIDO_MATERNO"] = df["CERTIFICADO"].astype(str) + "_" + df["APELLIDO_MATERNO"].astype(str)

def similitud_caracteres(a, b):
    a = limpiar_texto(a)
    b = limpiar_texto(b)
    if not a or not b:
        return 0.0
    return difflib.SequenceMatcher(None, a, b).ratio()

# =============================================================
# CUANDO SE SUBEN LOS ARCHIVOS
# =============================================================
if archivo_semana and archivo_bitacora:
    st.success("Archivos cargados correctamente. Procesando…")

    # ---------------------------------------------------------
    # CARGAR ARCHIVOS
    # ---------------------------------------------------------
    altas = pd.read_excel(archivo_semana, skiprows=2)
    hist_raw = pd.read_excel(archivo_bitacora)

    # TU CÓDIGO COMPLETO SE USA AQUÍ
    # (Cambié rutas por objetos cargados desde streamlit)

    # Renombrar columnas del histórico
    mapeo_columnas = {
        'NOMBRE': 'NOMBRE',
        'F. NAC': 'FECHA_DE_NACIMIENTO_BENEFICIARIO',
        'PARENTESCO': 'PARENTESCO',
        'APELLIDO PATERNO': 'APELLIDO_PATERNO',
        'APELLIDO MATERNO': 'APELLIDO_MATERNO',
        'CERT': 'CERTIFICADO'
    }

    hist_raw = hist_raw.rename(columns=mapeo_columnas)

    for col in hist_raw.columns:
        if hist_raw[col].dtype == "object":
            hist_raw[col] = hist_raw[col].apply(limpiar_texto)

    hist_raw["CERTIFICADO"] = hist_raw["CERTIFICADO"].apply(limpiar_certificado)
    hist_raw["FECHA_DE_NACIMIENTO_BENEFICIARIO"] = hist_raw["FECHA_DE_NACIMIENTO_BENEFICIARIO"].apply(limpiar_fecha_str)
    hist_raw["NOMBRE_COMPLETO"] = nombre_completo(hist_raw)

    hist = hist_raw.copy()

    # Procesar ALTAS igual que tu script
    altas.columns = altas.columns.str.replace(' ', '_').str.upper()
    cols = altas.columns.tolist()
    if len(cols) > 14:
        cols[2] = "FECHA_DE_NACIMIENTO_EMPLEADO"
        cols[14] = "FECHA_DE_NACIMIENTO_BENEFICIARIO"
    altas.columns = cols

    altas["FECHA_DE_MOVIMIENTO"] = pd.to_datetime(altas.get("FECHA_DE_MOVIMIENTO"), errors="coerce")
    fecha_limite = datetime.today() - relativedelta(months=6)
    altas_6 = altas[altas["FECHA_DE_MOVIMIENTO"] >= fecha_limite].copy()

    for col in ["NOMBRE", "APELLIDO_PATERNO", "APELLIDO_MATERNO", "PARENTESCO"]:
        if col in altas_6.columns:
            altas_6[col] = altas_6[col].apply(limpiar_texto)

    altas_6["CERTIFICADO"] = altas_6["CERTIFICADO"].apply(limpiar_certificado)
    altas_6["FECHA_DE_NACIMIENTO_BENEFICIARIO"] = altas_6["FECHA_DE_NACIMIENTO_BENEFICIARIO"].apply(limpiar_fecha_str)
    altas_6["NOMBRE_COMPLETO"] = nombre_completo(altas_6)
    altas_6["EDAD"] = altas_6["FECHA_DE_NACIMIENTO_BENEFICIARIO"].apply(calcular_edad)

    # Excluir mayores
    cond_hijo = (altas_6["PARENTESCO"].isin(["HIJO", "HIJA"])) & (altas_6["EDAD"] >= 21)
    cond_esposo = (altas_6["PARENTESCO"].isin(["ESPOSO", "ESPOSA"])) & (altas_6["EDAD"] >= 65)

    altas_6 = altas_6[~(cond_hijo | cond_esposo)]

    # Crear IDs
    crear_ids(altas_6)
    crear_ids(hist)

    # ---------------------------------------------------------
    #   CLASIFICAR
    # ---------------------------------------------------------
    H = {col: set(hist[col]) for col in hist.columns if col.startswith("CERTIFICADO")}

    def clasificar_mejorado(row):
        cert = row["CERTIFICADO"]
        nombre_id = f"{cert}_{row['NOMBRE']}"
        nomcomp_id = f"{cert}_{row['NOMBRE_COMPLETO']}"
        fecha_id = f"{cert}_{row['FECHA_DE_NACIMIENTO_BENEFICIARIO']}"
        ap_pat_id = f"{cert}_{row['APELLIDO_PATERNO']}"
        ap_mat_id = f"{cert}_{row['APELLIDO_MATERNO']}"

        nombre_match = nombre_id in H.get("CERTIFICADO_NOMBRE", set())
        nomcomp_match = nomcomp_id in H.get("CERTIFICADO_NOMBRE_COMPLETO", set())
        fecha_match = fecha_id in H.get("CERTIFICADO_FECHA", set())
        ap_pat_match = ap_pat_id in H.get("CERTIFICADO_APELLIDO_PATERNO", set())
        ap_mat_match = ap_mat_id in H.get("CERTIFICADO_APELLIDO_MATERNO", set())

        hist_cert = hist[hist["CERTIFICADO"] == cert]

        fuzzy_name_match = False
        if not hist_cert.empty:
            for h_nom in hist_cert["NOMBRE"].dropna().unique():
                if similitud_caracteres(row["NOMBRE"], h_nom) >= 0.8:
                    fuzzy_name_match = True
                    break

        if nombre_match and nomcomp_match and fecha_match:
            return "YA_EXISTE"
        if fuzzy_name_match:
            return "CAMBIO"
        if fecha_match:
            return "CAMBIO"
        if not (nombre_match or nomcomp_match or fecha_match):
            return "ALTA"
        return "REVISION"

    altas_6["TIPO"] = altas_6.apply(clasificar_mejorado, axis=1)

    # ---------------------------------------------------------
    # SEPARAR RESULTADOS
    # ---------------------------------------------------------
    altas_out = altas_6[altas_6["TIPO"] == "ALTA"]
    cambios_out = altas_6[altas_6["TIPO"] == "CAMBIO"]
    revision_out = altas_6[altas_6["TIPO"] == "REVISION"]
    ya_existe_out = altas_6[altas_6["TIPO"] == "YA_EXISTE"]

    # =============================================================
    # GRÁFICOS
    # =============================================================
    st.subheader("📊 Estadísticas")

    counts = {
        "ALTAS": len(altas_out),
        "CAMBIOS": len(cambios_out),
        "REVISION": len(revision_out)
    }

    # Pie chart
    fig1, ax1 = plt.subplots()
    ax1.pie(counts.values(), labels=counts.keys(), autopct="%1.1f%%")
    ax1.set_title("Distribución general")
    st.pyplot(fig1)

    # Barras
    fig2, ax2 = plt.subplots()
    ax2.bar(["Cambios", "Revisión"], [counts["CAMBIOS"], counts["REVISION"]])
    ax2.set_title("Cambios vs Revisión")
    st.pyplot(fig2)

    # =============================================================
    # DESCARGAS
    # =============================================================
    def df_to_download(df):
        output = BytesIO()
        df.to_excel(output, index=False)
        return output.getvalue()

    st.subheader("📥 Descargar resultados")

    st.download_button("📘 ALTAS", df_to_download(altas_out), "ALTAS.xlsx")
    st.download_button("📙 CAMBIOS", df_to_download(cambios_out), "CAMBIOS.xlsx")
    st.download_button("📗 REVISIÓN", df_to_download(revision_out), "REVISION.xlsx")
    st.download_button("📕 YA EXISTE", df_to_download(ya_existe_out), "YA_EXISTE.xlsx")

